<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/models/3DModels/DistanceCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile


zip_ref = zipfile.ZipFile('/content/drive/Shareddrives/Datasets SEFAI/fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
import plotly.express as px
import cv2
import mediapipe as mp

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import linspace
from mpl_toolkits import mplot3d
import glob, os
import json
import pandas as pd
import math

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

contour_list = [landmark_tuple for landmark_tuple in mp_face_mesh.FACEMESH_CONTOURS]
tass_list = [landmark_tuple for landmark_tuple in mp_face_mesh.FACEMESH_TESSELATION]
irises_list = [landmark_tuple for landmark_tuple in mp_face_mesh.FACEMESH_IRISES]
all_links_list = list(set(contour_list) | set(tass_list) | set(irises_list))
all_links_array = np.array(all_links_list)

In [ ]:
train_df = pd.read_csv("/content/drive/Shareddrives/Datasets SEFAI/training_set.csv")
test_df = pd.read_csv("/content/drive/Shareddrives/Datasets SEFAI/test_set.csv")
test_df['landmarks'] = test_df['landmarks'].apply(lambda lab: eval(lab))

train_df['landmarks'] = train_df['landmarks'].apply(lambda lab: eval(lab))
print(len(train_df))
print(len(test_df))

26705
6678


In [ ]:
train_dataset = np.empty(shape= (len(train_df),2626))
for i in range(len(train_df)):
  landmarks = train_df.iloc[i]["landmarks"]
  for j, couple in enumerate(all_links_array):
    x2 = landmarks[couple[1]][0]
    x1 = landmarks[couple[0]][0]
    y2 = landmarks[couple[1]][1]
    y1 = landmarks[couple[0]][1]
    z2 = landmarks[couple[1]][2]
    z1 = landmarks[couple[0]][2]
    distance =  math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    train_dataset[i][j] = distance


In [ ]:
test_dataset = np.empty(shape= (len(test_df),2626))
for i in range(len(test_df)):
  landmarks = test_df.iloc[i]["landmarks"]
  for j, couple in enumerate(all_links_array):
    x2 = landmarks[couple[1]][0]
    x1 = landmarks[couple[0]][0]
    y2 = landmarks[couple[1]][1]
    y1 = landmarks[couple[0]][1]
    z2 = landmarks[couple[1]][2]
    z1 = landmarks[couple[0]][2]
    distance =  math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    test_dataset[i][j] = distance

In [ ]:

label_dict = {"angry":0, "sad": 1, "neutral": 2, "surprise": 3, "disgust": 4, "fear": 5, "happy": 6}
test_df['label'] = test_df['label'].apply(lambda lab: label_dict[lab])

train_df['label'] = train_df['label'].apply(lambda lab: label_dict[lab])

y_test = test_df['label'].to_numpy()
y_train = train_df['label'].to_numpy()

In [ ]:
np.savez('distance_test_dataset.npz', data=test_dataset, labels=y_test)
np.savez('distance_train_dataset.npz', data=train_dataset, labels=y_train)

# Checkpoint to reload data


In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install -q mlflow
!databricks configure --host https://community.cloud.databricks.com/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
Username: gfesta24@gmail.com
Password: 
Repeat for confirmation: 


In [ ]:
from torch.utils.data import Dataset
import numpy as np
class CustomDataset(Dataset):
    def __init__(self, X_array, Y_array, transform=None):
        self.X = X_array
        self.Y = Y_array
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.Y[index]

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch
train_df = np.load('/content/drive/Shareddrives/Datasets SEFAI/distance_train_dataset.npz')
test_df = np.load('/content/drive/Shareddrives/Datasets SEFAI/distance_test_dataset.npz')

X_train= train_df['data']
X_test = test_df['data']
y_train = train_df['labels']
y_test = test_df['labels']

train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))

test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))


In [ ]:

def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

# Distance CNN

In [ ]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(128 * 326, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 7)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #print(x.shape)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        x = self.fc3(x)
        return x



In [ ]:
import mlflow
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/gfesta24@gmail.com/DistanceCNN")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/531884495140798', creation_time=1688071348937, experiment_id='531884495140798', last_update_time=1688071348937, lifecycle_stage='active', name='/Users/gfesta24@gmail.com/DistanceCNN', tags={'mlflow.experiment.sourceName': '/Users/gfesta24@gmail.com/DistanceCNN',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'gfesta24@gmail.com',
 'mlflow.ownerId': '1923923806180228'}>

In [ ]:
from sklearn.model_selection import ParameterGrid
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

param_grid = {'batch_size': [64, 128, 256, 512],'lr': [0.0001, 0.001], "decay":[0, 0.001, 0.01]}
expanded_grid = ParameterGrid(param_grid)
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("/Users/gfesta24@gmail.com/DistanceCNN")

for i in range(len(expanded_grid)):

  runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id], filter_string=" and ".join([f"params.{k} = '{v}'" for k, v in expanded_grid[i].items()]))
  if not len(runs) == 0:
    print("RUN: ", [f"params.{k} = '{v}'" for k, v in expanded_grid[i].items()], " già completata" )
  else:
    train_loader = DataLoader(train_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=True)

    test_loader = DataLoader(test_dataset, batch_size=expanded_grid[i]['batch_size'], shuffle=False)

    model = CNN()
    optimizer = optim.Adam(model.parameters(), lr=expanded_grid[i]['lr'], weight_decay=expanded_grid[i]['decay'])
    criterion = nn.CrossEntropyLoss()
    model.to(device)

    mlflow.start_run()

    mlflow.set_tag("model_name", "3LayerCNN")
    mlflow.log_param("lr", expanded_grid[i]['lr'])
    mlflow.log_param("batch_size", expanded_grid[i]['batch_size'])
    mlflow.log_param("decay", expanded_grid[i]['decay'])

    patience=3
    counter = 0
    stop = False

    # Training loop
    best_accuracy = 0.0
    best_loss = 1000

    for epoch in range(100):
      print(counter)
      if stop:
        print(stop)
        break
      model.train()
      running_loss = 0.0
      running_acc = 0
      tot_seen = 0
      for X_batch, y_batch in train_loader:
        X_batch=X_batch.unsqueeze(1)
        X_batch = X_batch.type(torch.cuda.FloatTensor)
        y_batch = y_batch.long()
        #manda i batch al device
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        #azzera gradiente
        optimizer.zero_grad()
        #predict
        y_pred = model(X_batch)


        #print(y_batch.shape)
        loss = criterion(y_pred, y_batch)
        _,acc = accuracy(y_pred, y_batch)
        #backpropagation della loss
        loss.backward()
        #ottimizzazione
        optimizer.step()

        #somma della loss e dell'accuracy per il batch
        running_loss += loss.item()
        running_acc += acc
        tot_seen += len(y_batch)

      train_accuracy = running_acc/tot_seen
      train_loss = running_loss / len(train_loader)
      print(f'Epoch {epoch}: | Loss: {train_loss:.5f} | Acc: {running_acc/tot_seen:.3f}')
      mlflow.log_metric("train_loss", train_loss, step=epoch)
      mlflow.log_metric("train_acc", train_accuracy, step=epoch)
      model.eval()

      tot_corrette = 0
      tot_eseguite = 0
      running_test_loss = 0
      val_loss = 0

      with torch.no_grad():

        for images, labels in test_loader:
            images=images.unsqueeze(1)
            images = images.to(device)
            labels = labels.long()
            labels = labels.to(device)

            outputs = model(images)
            test_loss = criterion(outputs, labels)
            _, n_corrette=accuracy(outputs, labels)

            running_test_loss += test_loss.item()
            tot_corrette+=n_corrette.item()
            tot_eseguite+=labels.shape[0]

        test_acc=100* (tot_corrette/tot_eseguite)
        test_loss = running_test_loss / len(test_loader)
        print("Test acc: ", test_acc)
        print("Test loss: ", test_loss)
        mlflow.log_metric("test_acc", test_acc, step=epoch)
        mlflow.log_metric("test_loss", test_loss, step=epoch)

        if test_loss < best_loss:
              print("MIGLIORATO")
              torch.save(model.state_dict(), 'model_weights.pth')
              best_loss = test_loss
              best_model_train_acc=train_accuracy
              best_model_test_acc=test_acc
              best_model_test_loss=test_loss
              best_model_train_loss=running_loss / len(train_loader)
              counter = 0
              # Salva i pesi del modello se la validation loss è migliorata
              torch.save(model.state_dict(), 'best_model.pt')
        else:
            counter += 1
            # Verifica se raggiunto il criterio di early stopping
            if counter >= patience:
                  print(f'Early stopping at epoch {epoch+1}')
                  mlflow.set_tag("Epochs_stopped", epoch+1)
                  mlflow.log_artifact("best_model.pt")
                  mlflow.log_metric("best_test_acc", best_model_test_acc)
                  mlflow.log_metric("best_test_loss", best_model_test_loss)
                  mlflow.log_metric("best_train_acc", best_model_train_acc)
                  mlflow.log_metric("best_train_loss", best_model_train_loss)
                  mlflow.end_run()
                  stop=True
        print("BEST TEST LOSS: ", best_loss)




0
Epoch 0: | Loss: 1.80826 | Acc: 0.260
Test acc:  25.666367175801135
Test loss:  1.8039232605979556
MIGLIORATO
BEST TEST LOSS:  1.8039232605979556
0
Epoch 1: | Loss: 1.80370 | Acc: 0.260
Test acc:  25.666367175801135
Test loss:  1.8047355504263014
BEST TEST LOSS:  1.8039232605979556
1
Epoch 2: | Loss: 1.80280 | Acc: 0.260
Test acc:  25.666367175801135
Test loss:  1.800826300893511
MIGLIORATO
BEST TEST LOSS:  1.800826300893511
0
Epoch 3: | Loss: 1.79882 | Acc: 0.260
Test acc:  25.666367175801135
Test loss:  1.788718741280692
MIGLIORATO
BEST TEST LOSS:  1.788718741280692
0
Epoch 4: | Loss: 1.68414 | Acc: 0.317
Test acc:  34.17190775681342
Test loss:  1.6107831969147637
MIGLIORATO
BEST TEST LOSS:  1.6107831969147637
0
Epoch 5: | Loss: 1.52761 | Acc: 0.404
Test acc:  42.108415693321355
Test loss:  1.4993064448946998
MIGLIORATO
BEST TEST LOSS:  1.4993064448946998
0
Epoch 6: | Loss: 1.46716 | Acc: 0.427
Test acc:  43.06678646301288
Test loss:  1.4608079893248422
MIGLIORATO
BEST TEST LOSS:  